In [1]:
from pandas_datareader import data
import pandas as pd
import pandas_datareader.data as web
from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import os
import re

In [2]:
linkedin = pd.read_csv('temp_datalab_records_linkedin_company.csv', low_memory=False)

In [3]:
linkedin.head()

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,website,entity_id,cusip,isin
0,58329,2015-09-14,Goldman Sachs,552254,38124,https://www.linkedin.com/company/1382,Investment Banking,2015-09-14 00:00:00+00,2015-09-14 00:00:00+00,NaN,NaN,NaN,NaN,NaN
1,58329,2015-09-15,Goldman Sachs,552862,38141,https://www.linkedin.com/company/1382,Investment Banking,2015-09-15 00:00:00+00,2015-09-15 00:00:00+00,NaN,NaN,NaN,NaN,NaN
2,58363,2015-09-16,United Technologies,59157,14982,https://www.linkedin.com/company/2426,Aviation & Aerospace,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
3,58366,2015-09-16,Novo Nordisk,336175,26448,https://www.linkedin.com/company/2227,Pharmaceuticals,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
4,58371,2015-09-16,"Lowe's Companies, Inc.",134255,62574,https://www.linkedin.com/company/4128,Retail,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN


In [4]:
companies = linkedin.company_name.unique()

In [5]:
companies.size

5028

In [6]:
# Find companies where hiring is most predictive of stock prices.
# function for getting stock prices for a given company
def stock(symbol, start, end):
    # get the stock
    f = web.DataReader(symbol, 'iex', start, end)
    return f

# function for pulling out company linkedin data
def companyli(company):
    df = linkedin[linkedin.company_name==company]
    return df[['as_of_date','employees_on_platform']]

# function for getting the symbol
def symbol(name):
    symbols = get_nasdaq_symbols()
    symbols = symbols[symbols['Security Name'].str.contains(name)]
    symbols = list(symbols.index.values)
    symbols = list(filter(lambda x: '$' not in x, symbols))
    return symbols

# find the difference in the stock price over time
def diffprice(cs):
    # using closing price
    closing = cs.close
    closing.index = pd.to_datetime(closing.index, format='%Y-%m-%d')
    closing = closing.resample('M').first()
    diffclose = pd.DataFrame(closing.diff())
    # return monthly stock price change in percent total from previous month
    diffpercent = diffclose.close[1:].to_numpy()/closing[0:-1].to_numpy()
    diffclose['percentstock'] = np.append(np.nan, diffpercent)
    return diffclose

# find the difference in # of employees over time
def diffemployees(cli):
    cli.as_of_date = pd.to_datetime(cli.as_of_date, format='%Y-%m-%d')
    cli = cli.set_index('as_of_date')
    employee = cli.resample('M').first()
    diffemp = employee.diff()
    # return montly employee number change in percent total from previous month
    diffpercent = diffemp.employees_on_platform[1:].to_numpy()/employee.employees_on_platform[0:-1].to_numpy()
    diffemp['percentemp'] = np.append(np.nan, diffpercent)
    return diffemp

# same time frame
def timerange(ediff):
    start = ediff.index.min()
    end = ediff.index.max()
    return start, end

# # plot change in # of employees to stock price change
# def plotchange(sdiff, ediff):
#     scaling = ediff.max()/sdiff.max()
#     scaledediff = ediff/scaling
#     plt.plot(sdiff)
#     plt.plot(scaledediff)
#     plt.show()
    
# def linregress(sdiff, ediff):
def linregress(x, y):
    lr = LinearRegression()
    lr.fit(x, y)
    return lr.score(x, y)
# why did the industry change?

In [7]:
# get symbols
compdict = dict.fromkeys(companies)
for k in companies:
    try:
        i = re.search('[^a-zA-Z ]', k).start()
        k = k[:i].rstrip()
    except:
        pass
    s = symbol(k)
    compdict[k] = s

In [8]:
# select one symbol
compdict['Goldman Sachs'] = ['GS']

In [9]:
# look at each company and calculate the linear regression

r2 = np.zeros([companies.shape[0], 2])
validcomp = []
c = 0
totmem = 0
for k in compdict.keys():
    if (compdict[k] is not None) and (len(compdict[k]) == 1):
        # get employee count from linkedin
        emp = companyli(k)
        if emp.shape[0] > 0:
            diffemp = diffemployees(emp)

            # get the timerange
            start, end = timerange(diffemp)

            # get stock difference
            sym = compdict[k][0]
            st = stock(sym, start, end)
            totmem += st.memory_usage().sum()
            diffst = diffprice(st)

            # remove nan from the employee data
            y = diffemp.percentemp[1:]
            # remove nan from the stock data
            x = diffst.percentstock[1:]

            # at least half a year of data
            if (x.shape[0] > 5) and (y.isnull().values.sum() == 0):
                # sometimes stock data is smaller than the employee data
                y = y.loc[x.index.min():x.index.max()]

                # get r2 value
#                 r2[c] = linregress(x, y)
                # stock to predict employee change
                r2[c, 0] = linregress(x[0:-1].values.reshape([-1,1]), y[1:].values.reshape([-1,1]))
                # employee to predict stock change
                r2[c, 1] = linregress(y[0:-1].values.reshape([-1,1]), x[1:].values.reshape([-1,1]))
                # get the name of the company
                validcomp.append(k)
                c += 1
                
                # save
                df = y.to_frame()
                df = df.join(x)
                df.to_csv('deltastem/' + k +'.csv', index=True)

/Users/leey/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [10]:
# convert from bytes to megabytes
(totmem+linkedin.memory_usage().sum())*10**-6

304.652816

In [11]:
# save
df = pd.DataFrame(validcomp, columns=['names'])
df['stpredem'] = r2[:c, 0]
df['empredst'] = r2[:c, 1]
df.to_csv('deltastemr2.csv', index=False)